In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.5/116.5 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21484 sha256=40d687d657da6db862a174e5969cbbd77f1ea0fc301b86010cd8884fed7b3ba0
  Stored in directory: /Users/cristianosantosdesouza/Library/Caches/pip/wheels/e7/e6/28/864bdfee5339dbd6ddcb5a186286a8e217648ec198bdf0097d
Successfully built promise


In [2]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

2024-05-04 07:05:47.901439: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version:  2.16.1
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


In [6]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [02:29<00:00, 142.62s/ url]
Dl Size...: 81 MiB [02:29,  1.81s/ MiB]
Dl Completed...: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [02:32<00:00, 142.62s/ url]
Dl Size...: 82 MiB [02:32,  1.97s/ MiB]
Dl Completed...: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [02:34<00:00, 142.62s/ url]
Dl Size...: 83 MiB [02:34,  2.03s/ MiB]
Dl Completed...: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [02:35<00:00, 142.62s/ url]
Dl Size...: 84 MiB [02:35,  1.93s/ MiB]
Dl Completed...: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [02:37<00:00, 142.62s/ url]
Dl Size...: 85 MiB [02:37,  1.88s/ MiB]
Dl Completed...: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [02:39<00:00, 142.62

RecursionError: maximum recursion depth exceeded while calling a Python object

In [7]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

NameError: name 'train_data' is not defined

In [8]:
train_labels_batch

NameError: name 'train_labels_batch' is not defined

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))